In [1]:
from __future__ import print_function, division
from builtins import range
import numpy as np

## make Gridworld

In [2]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
ACTION_SPACE = ('U', 'D', 'L', 'R')


class Grid:  # Environment

    def __init__(self,rows,cols,start):
        self.rows = rows
        self.cols = cols
        self.i = start[0]
        self.j = start[1]

    def set(self, rewards, actions):

        self.rewards = rewards
        self.actions = actions

    def set_state(self, s):
        self.i = s[0]
        self.j = s[1]

    def current_state(self):
        return (self.i, self.j)

    def is_terminal(self, s):
        return s not in self.actions

    def get_next_state(self, s, a):

    # this answers: where would I end up if I perform action 'a' in state 's'?

        (i, j) = (s[0], s[1])

    # if this action moves you somewhere else, then it will be in this dictionary

        if a in self.actions[(i, j)]:
            if a == 'U':
                i -= 1
            elif a == 'D':
                i += 1
            elif a == 'R':
                j += 1
            elif a == 'L':
                j -= 1
        return (i, j)

    def all_states(self):
        return set(self.actions.keys()) | set(self.rewards.keys())





## Define Grid 3*4

In [3]:
def standard_grid():

  # .  .  .  1
  # .  x  . -1
  # s  .  .  .

    grid = Grid(3, 4, (2, 0))
    rewards = {(0, 3): 1, (1, 3): -1}
    actions = {
        (0, 0): ('D', 'R'),
        (0, 1): ('L', 'R'),
        (0, 2): ('L', 'D', 'R'),
        (1, 0): ('U', 'D'),
        (1, 2): ('U', 'D', 'R'),
        (2, 0): ('U', 'R'),
        (2, 1): ('L', 'R'),
        (2, 2): ('L', 'R', 'U'),
        (2, 3): ('L', 'U'),
        }
    grid.set(rewards, actions)
    return grid

## Display Grid

In [4]:
def print_values(V, g):
    for i in range(g.rows):
        print("---------------------------")
        for j in range(g.cols):
            v = V.get((i, j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")  
        print("")


def print_policy(P, g):
    for i in range(g.rows):
        print("---------------------------")
        for j in range(g.cols):
            a = P.get((i, j), " ")
            print("  %s  |" % a, end="")
        print("")


## evaluate fix policy and find optimal value functions

In [5]:
grid = standard_grid()

### fixed policy ###
policy = {}
for state in grid.actions.keys():
    policy[state] = np.random.choice(ACTION_SPACE)
print_policy(policy, grid)


    

---------------------------
  U  |  R  |  R  |     |
---------------------------
  L  |     |  L  |     |
---------------------------
  D  |  R  |  L  |  U  |


In [6]:
# initialize V(s) = 0
V = {}
for s in grid.all_states():
    V[s] = 0

print_values(V, grid)

---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|


## Reward and Transition probability

In [7]:
transition_probs = {}
rewards = {}

for row in range(grid.rows):
    for col in range(grid.cols):
        state = (row, col)
        if not grid.is_terminal(state):
            for action in ACTION_SPACE:
                next_state = grid.get_next_state(state, action)
                transition_probs[(state, action, next_state)] = 1
                if next_state in grid.rewards:
                    rewards[(state, action, next_state)] = grid.rewards[next_state]


print(rewards)
print("---------------")
print(transition_probs)

{((0, 2), 'R', (0, 3)): 1, ((1, 2), 'R', (1, 3)): -1, ((2, 3), 'U', (1, 3)): -1}
---------------
{((0, 0), 'U', (0, 0)): 1, ((0, 0), 'D', (1, 0)): 1, ((0, 0), 'L', (0, 0)): 1, ((0, 0), 'R', (0, 1)): 1, ((0, 1), 'U', (0, 1)): 1, ((0, 1), 'D', (0, 1)): 1, ((0, 1), 'L', (0, 0)): 1, ((0, 1), 'R', (0, 2)): 1, ((0, 2), 'U', (0, 2)): 1, ((0, 2), 'D', (1, 2)): 1, ((0, 2), 'L', (0, 1)): 1, ((0, 2), 'R', (0, 3)): 1, ((1, 0), 'U', (0, 0)): 1, ((1, 0), 'D', (2, 0)): 1, ((1, 0), 'L', (1, 0)): 1, ((1, 0), 'R', (1, 0)): 1, ((1, 2), 'U', (0, 2)): 1, ((1, 2), 'D', (2, 2)): 1, ((1, 2), 'L', (1, 2)): 1, ((1, 2), 'R', (1, 3)): 1, ((2, 0), 'U', (1, 0)): 1, ((2, 0), 'D', (2, 0)): 1, ((2, 0), 'L', (2, 0)): 1, ((2, 0), 'R', (2, 1)): 1, ((2, 1), 'U', (2, 1)): 1, ((2, 1), 'D', (2, 1)): 1, ((2, 1), 'L', (2, 0)): 1, ((2, 1), 'R', (2, 2)): 1, ((2, 2), 'U', (1, 2)): 1, ((2, 2), 'D', (2, 2)): 1, ((2, 2), 'L', (2, 1)): 1, ((2, 2), 'R', (2, 3)): 1, ((2, 3), 'U', (1, 3)): 1, ((2, 3), 'D', (2, 3)): 1, ((2, 3), 'L', (2, 

## Policy evaluation using bootstrapping

In [8]:
def evaluate_deterministic_policy(grid, policy, V=None):
    iteration = 0
    while True:
        for state in grid.all_states():
            if not grid.is_terminal(state):
                new_v = 0
                for action in ACTION_SPACE:
                    for next_state in grid.all_states():
                        #deteministic actions:
                        action_prob = 1 if policy.get(state) == action else 0

                        r = rewards.get((state, action, next_state), 0)
                        new_v += action_prob * transition_probs.get((state, action, next_state), 0) * (r + 0.9 * V[next_state])
                        

                V[state] = new_v



        iteration = iteration+1
    #     print_values(V, grid)
    #     print("\n")
        if iteration == 100:
            break

    
    return V


In [9]:
policy

{(0, 0): 'U',
 (0, 1): 'R',
 (0, 2): 'R',
 (1, 0): 'L',
 (1, 2): 'L',
 (2, 0): 'D',
 (2, 1): 'R',
 (2, 2): 'L',
 (2, 3): 'U'}

In [ ]:
itteration = 0

# ### fixed policy ###
# policy = {
# (2, 0): 'U',
# (1, 0): 'U',
# (0, 0): 'R',
# (0, 1): 'R',
# (0, 2): 'R',
# (1, 2): 'R',# this is not optimal
# (2, 1): 'R',
# (2, 2): 'U',
# (2, 3): 'L',
# (1, 1): 'X',
# (1, 3): 'H',
# (0, 3): 'G'
# }
is_policy_converged = False
while True:
    V = evaluate_deterministic_policy(grid, policy, V)
    for state in grid.actions.keys():
        best_value = float('-inf')
        for action in ACTION_SPACE:
            v = 0
            for next_state in grid.all_states():
                r = rewards.get((state, action, next_state), 0)
                v += transition_probs.get((state, action, next_state), 0) * (r + 0.9 * V[next_state])
            if v > best_value:
                best_action = action
                best_value = v
    
        if best_action != policy[state]:
            is_policy_converged = False
        policy[state] = best_action
    if is_policy_converged:
        break
print_policy(policy, grid)